# Fase 3: Entrenamiento y Evaluación de Modelos
# Marketing Campaign Response Prediction

---

## Objetivo

Este notebook implementa el entrenamiento y evaluación de múltiples modelos de Machine Learning:

1. **Carga de datos**: Datos transformados de la Fase 2
2. **Entrenamiento de modelos**: 7 algoritmos diferentes
3. **Evaluación**: Métricas completas (accuracy, precision, recall, F1, ROC-AUC)
4. **Validación cruzada**: 5-fold estratificada
5. **Comparación de modelos**: Tabla comparativa y visualizaciones
6. **Selección del mejor modelo**: Basado en métricas y consistencia
7. **Guardado del modelo**: Modelo final listo para despliegue

---

## 1. Importación de Librerías

In [ ]:
# Manipulación de datos
import pandas as pd
import numpy as np
import json
from datetime import datetime

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning - Scikit-learn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score, cross_validate, StratifiedKFold
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, 
    roc_auc_score, roc_curve, auc, confusion_matrix, 
    classification_report, average_precision_score
)

# XGBoost y LightGBM
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Model persistence
import joblib

# Configuración
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette('husl')
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.4f}'.format)

print('✅ Librerías importadas correctamente')

## 2. Carga de Datos Transformados

In [ ]:
print('\n' + '='*80)
print('CARGA DE DATOS')
print('='*80)

In [ ]:
# Cargar datos transformados de la Fase 2
# Opción 1: Cargar desde archivos CSV (recomendado)
try:
    X_train = pd.read_csv('../../X_train_transformed.csv')
    X_test = pd.read_csv('../../X_test_transformed.csv')
    y_train = pd.read_csv('../../y_train.csv', squeeze=True)
    y_test = pd.read_csv('../../y_test.csv', squeeze=True)
    
    # Convertir a numpy arrays
    X_train = X_train.values
    X_test = X_test.values
    y_train = y_train.values
    y_test = y_test.values
    
    print('✅ Datos cargados desde archivos CSV')
    print(f'   X_train: {X_train.shape}')
    print(f'   X_test: {X_test.shape}')
    print(f'   y_train: {y_train.shape}')
    print(f'   y_test: {y_test.shape}')
    
except FileNotFoundError as e:
    print(f'❌ Error: No se encontraron los archivos de la Fase 2')
    print(f'   Asegúrate de haber ejecutado la Fase 2 primero')
    print(f'   Archivos necesarios:')
    print(f'     - X_train_transformed.csv')
    print(f'     - X_test_transformed.csv')
    print(f'     - y_train.csv')
    print(f'     - y_test.csv')
    raise e

In [ ]:
# Verificar distribución de clases
print('\nDistribución de clases:')
print(f'Train - No acepta (0): {(y_train == 0).sum()} ({(y_train == 0).mean()*100:.2f}%)')
print(f'Train - Acepta (1):    {(y_train == 1).sum()} ({(y_train == 1).mean()*100:.2f}%)')
print(f'Test - No acepta (0):  {(y_test == 0).sum()} ({(y_test == 0).mean()*100:.2f}%)')
print(f'Test - Acepta (1):     {(y_test == 1).sum()} ({(y_test == 1).mean()*100:.2f}%)')

In [ ]:
# Cargar preprocessor (opcional, para obtener nombres de features)
try:
    preprocessor = joblib.load('../../preprocessor.pkl')
    print('\n✅ Preprocessor cargado')
except FileNotFoundError:
    print('\n⚠️ Preprocessor no encontrado (no crítico)')
    preprocessor = None

---
## 3. DEFINICIÓN DE MODELOS

In [ ]:
print('\n' + '='*80)
print('DEFINICIÓN DE MODELOS')
print('='*80)

In [ ]:
# Calcular peso para balanceo de clases
n_classes_0 = (y_train == 0).sum()
n_classes_1 = (y_train == 1).sum()
scale_pos_weight = n_classes_0 / n_classes_1

print(f'\nBalance de clases:')
print(f'  Clase 0 (No acepta): {n_classes_0}')
print(f'  Clase 1 (Acepta):    {n_classes_1}')
print(f'  Ratio: {scale_pos_weight:.2f}:1')

In [ ]:
# Definir modelos
models = {
    'Logistic Regression': LogisticRegression(
        max_iter=1000, 
        random_state=42, 
        class_weight='balanced'
    ),
    'Random Forest': RandomForestClassifier(
        n_estimators=100, 
        random_state=42, 
        class_weight='balanced', 
        n_jobs=-1
    ),
    'Gradient Boosting': GradientBoostingClassifier(
        n_estimators=100, 
        random_state=42
    ),
    'Extra Trees': ExtraTreesClassifier(
        n_estimators=100, 
        random_state=42, 
        class_weight='balanced', 
        n_jobs=-1
    ),
    'SVM': SVC(
        kernel='rbf', 
        probability=True, 
        random_state=42, 
        class_weight='balanced'
    ),
    'XGBoost': XGBClassifier(
        n_estimators=100, 
        random_state=42, 
        scale_pos_weight=scale_pos_weight,
        use_label_encoder=False, 
        eval_metric='logloss'
    ),
    'LightGBM': LGBMClassifier(
        n_estimators=100, 
        random_state=42, 
        class_weight='balanced', 
        verbose=-1
    )
}

print(f'\n✅ {len(models)} modelos definidos:')
for name in models.keys():
    print(f'  - {name}')

---
## 4. FUNCIÓN DE ENTRENAMIENTO Y EVALUACIÓN

In [ ]:
def build_model(model, X_train, y_train, X_test, y_test, model_name="Model", cv_folds=5):
    """
    Entrena y evalúa un modelo de clasificación.
    """
    print(f'\n{"="*80}')
    print(f'ENTRENANDO: {model_name}')
    print(f'{"="*80}')
    
    # Entrenar modelo
    start_time = datetime.now()
    model.fit(X_train, y_train)
    training_time = (datetime.now() - start_time).total_seconds()
    
    print(f'✅ Modelo entrenado en {training_time:.2f} segundos')
    
    # Predicciones
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Probabilidades
    try:
        y_train_proba = model.predict_proba(X_train)[:, 1]
        y_test_proba = model.predict_proba(X_test)[:, 1]
    except AttributeError:
        y_train_proba = None
        y_test_proba = None
    
    # Métricas en train
    train_metrics = {
        'accuracy': accuracy_score(y_train, y_train_pred),
        'precision': precision_score(y_train, y_train_pred, zero_division=0),
        'recall': recall_score(y_train, y_train_pred, zero_division=0),
        'f1': f1_score(y_train, y_train_pred, zero_division=0)
    }
    
    if y_train_proba is not None:
        train_metrics['roc_auc'] = roc_auc_score(y_train, y_train_proba)
    
    # Métricas en test
    test_metrics = {
        'accuracy': accuracy_score(y_test, y_test_pred),
        'precision': precision_score(y_test, y_test_pred, zero_division=0),
        'recall': recall_score(y_test, y_test_pred, zero_division=0),
        'f1': f1_score(y_test, y_test_pred, zero_division=0)
    }
    
    if y_test_proba is not None:
        test_metrics['roc_auc'] = roc_auc_score(y_test, y_test_proba)
    
    # Validación cruzada
    cv_scores = {}
    if cv_folds > 0:
        cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=42)
        scoring = ['accuracy', 'precision', 'recall', 'f1', 'roc_auc']
        cv_results = cross_validate(model, X_train, y_train, cv=cv, scoring=scoring, n_jobs=-1)
        
        for metric in scoring:
            cv_scores[f'{metric}_mean'] = cv_results[f'test_{metric}'].mean()
            cv_scores[f'{metric}_std'] = cv_results[f'test_{metric}'].std()
    
    # Overfitting check
    overfitting = {
        'accuracy_diff': train_metrics['accuracy'] - test_metrics['accuracy'],
        'f1_diff': train_metrics['f1'] - test_metrics['f1']
    }
    
    # Imprimir métricas
    print(f'\n📊 Métricas en Test:')
    print(f'   Accuracy:  {test_metrics["accuracy"]:.4f}')
    print(f'   Precision: {test_metrics["precision"]:.4f}')
    print(f'   Recall:    {test_metrics["recall"]:.4f}')
    print(f'   F1-Score:  {test_metrics["f1"]:.4f}')
    if 'roc_auc' in test_metrics:
        print(f'   ROC-AUC:   {test_metrics["roc_auc"]:.4f}')
    
    if cv_folds > 0:
        print(f'\n📊 Validación Cruzada ({cv_folds}-fold):')
        print(f'   F1-Score: {cv_scores["f1_mean"]:.4f} ± {cv_scores["f1_std"]:.4f}')
        print(f'   ROC-AUC:  {cv_scores["roc_auc_mean"]:.4f} ± {cv_scores["roc_auc_std"]:.4f}')
    
    print(f'\n⚠️ Overfitting Check:')
    print(f'   Accuracy diff: {overfitting["accuracy_diff"]:.4f}')
    print(f'   F1 diff:       {overfitting["f1_diff"]:.4f}')
    
    if overfitting['f1_diff'] > 0.1:
        print(f'   ⚠️ Posible overfitting detectado')
    else:
        print(f'   ✅ Modelo generaliza bien')
    
    return {
        'model': model,
        'model_name': model_name,
        'y_train_pred': y_train_pred,
        'y_test_pred': y_test_pred,
        'y_train_proba': y_train_proba,
        'y_test_proba': y_test_proba,
        'train_metrics': train_metrics,
        'test_metrics': test_metrics,
        'cv_scores': cv_scores,
        'overfitting': overfitting,
        'training_time': training_time
    }

print('✅ Función build_model definida')

---
## 5. ENTRENAMIENTO DE MODELOS

In [ ]:
print('\n' + '='*80)
print('ENTRENAMIENTO DE MÚLTIPLES MODELOS')
print('='*80)
print(f'Train set: {X_train.shape}')
print(f'Test set:  {X_test.shape}')
print(f'CV folds:  5')

In [ ]:
# Entrenar todos los modelos
results = {}
cv_folds = 5

for model_name, model in models.items():
    try:
        result = build_model(
            model, X_train, y_train, X_test, y_test,
            model_name=model_name, cv_folds=cv_folds
        )
        results[model_name] = result
    except Exception as e:
        print(f'\n❌ Error entrenando {model_name}: {str(e)}')
        continue

print(f'\n{"="*80}')
print(f'✅ Entrenamiento completado: {len(results)} modelos')
print(f'{"="*80}')

---
## 6. COMPARACIÓN DE MODELOS

In [ ]:
print('\n' + '='*80)
print('COMPARACIÓN DE MODELOS')
print('='*80)

In [ ]:
# Crear DataFrame comparativo
comparison_data = []

for model_name, result in results.items():
    test_metrics = result['test_metrics']
    cv_scores = result.get('cv_scores', {})
    
    row = {
        'Model': model_name,
        'Accuracy': test_metrics['accuracy'],
        'Precision': test_metrics['precision'],
        'Recall': test_metrics['recall'],
        'F1-Score': test_metrics['f1'],
        'ROC-AUC': test_metrics.get('roc_auc', np.nan),
        'CV_F1_mean': cv_scores.get('f1_mean', np.nan),
        'CV_F1_std': cv_scores.get('f1_std', np.nan),
        'CV_ROC-AUC_mean': cv_scores.get('roc_auc_mean', np.nan),
        'Overfitting_F1': result['overfitting']['f1_diff'],
        'Training_Time': result['training_time']
    }
    comparison_data.append(row)

df_comparison = pd.DataFrame(comparison_data)
df_comparison = df_comparison.sort_values('F1-Score', ascending=False)

print('\n📊 Tabla Comparativa (ordenada por F1-Score):\n')
print(df_comparison.to_string(index=False))

In [ ]:
# Identificar mejor modelo
best_model_name = df_comparison.iloc[0]['Model']
best_result = results[best_model_name]

print(f'\n🏆 Mejor Modelo: {best_model_name}')
print(f'   F1-Score: {df_comparison.iloc[0]["F1-Score"]:.4f}')
print(f'   ROC-AUC:  {df_comparison.iloc[0]["ROC-AUC"]:.4f}')
print(f'   Accuracy: {df_comparison.iloc[0]["Accuracy"]:.4f}')

---
## 7. VISUALIZACIONES

In [ ]:
# Gráfico 1: Comparación de métricas principales
fig, ax = plt.subplots(figsize=(12, 6))

metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
df_plot = df_comparison[['Model'] + metrics_to_plot].set_index('Model')

df_plot.plot(kind='bar', ax=ax, rot=45)
ax.set_title('Comparación de Métricas Principales', fontsize=14, fontweight='bold')
ax.set_ylabel('Score')
ax.legend(loc='lower right')
ax.grid(axis='y', alpha=0.3)
ax.set_ylim([0, 1])

plt.tight_layout()
plt.savefig('../../model_comparison_metrics.png', dpi=300, bbox_inches='tight')
print('✅ Gráfico guardado: model_comparison_metrics.png')
plt.show()

In [ ]:
# Gráfico 2: Comparación de ROC-AUC
fig, ax = plt.subplots(figsize=(10, 6))

df_comparison_sorted = df_comparison.sort_values('ROC-AUC', ascending=True)
ax.barh(df_comparison_sorted['Model'], df_comparison_sorted['ROC-AUC'], color='skyblue')
ax.set_title('Comparación de ROC-AUC', fontsize=14, fontweight='bold')
ax.set_xlabel('ROC-AUC Score')
ax.grid(axis='x', alpha=0.3)
ax.set_xlim([0, 1])

plt.tight_layout()
plt.savefig('../../model_comparison_roc_auc.png', dpi=300, bbox_inches='tight')
print('✅ Gráfico guardado: model_comparison_roc_auc.png')
plt.show()

In [ ]:
# Gráfico 3: Curvas ROC
plt.figure(figsize=(10, 8))

colors = ['blue', 'red', 'green', 'orange', 'purple', 'brown', 'pink']

for idx, (model_name, result) in enumerate(results.items()):
    if result['y_test_proba'] is not None:
        fpr, tpr, _ = roc_curve(y_test, result['y_test_proba'])
        roc_auc = auc(fpr, tpr)
        
        plt.plot(fpr, tpr, color=colors[idx % len(colors)], lw=2,
                label=f'{model_name} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2, label='Random (AUC = 0.500)')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('Curvas ROC - Comparación de Modelos', fontsize=14, fontweight='bold')
plt.legend(loc='lower right')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.savefig('../../roc_curves_comparison.png', dpi=300, bbox_inches='tight')
print('✅ Gráfico guardado: roc_curves_comparison.png')
plt.show()

In [ ]:
# Gráfico 4: Matrices de confusión
n_models = len(results)
n_cols = 3
n_rows = (n_models + n_cols - 1) // n_cols

fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 5*n_rows))
axes = axes.ravel() if n_models > 1 else [axes]

for idx, (model_name, result) in enumerate(results.items()):
    cm = confusion_matrix(y_test, result['y_test_pred'])
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[idx],
               cbar=False, square=True, linewidths=1, linecolor='black')
    axes[idx].set_title(f'{model_name}', fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('Real')
    axes[idx].set_xlabel('Predicho')
    axes[idx].set_xticklabels(['No (0)', 'Sí (1)'])
    axes[idx].set_yticklabels(['No (0)', 'Sí (1)'])

# Ocultar ejes sobrantes
for idx in range(n_models, len(axes)):
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('../../confusion_matrices.png', dpi=300, bbox_inches='tight')
print('✅ Gráfico guardado: confusion_matrices.png')
plt.show()

In [ ]:
# Gráfico 5: Feature Importance (solo para modelos que lo soporten)
if hasattr(best_result['model'], 'feature_importances_'):
    importances = best_result['model'].feature_importances_
    
    # Obtener nombres de features si están disponibles
    try:
        # Intentar cargar desde archivo CSV
        feature_names = list(pd.read_csv('../../X_train_transformed.csv', nrows=0).columns)
    except:
        feature_names = [f'Feature_{i}' for i in range(len(importances))]
    
    # Top 20 features
    top_n = 20
    feature_importance_df = pd.DataFrame({
        'feature': feature_names,
        'importance': importances
    }).sort_values('importance', ascending=False).head(top_n)
    
    plt.figure(figsize=(10, 8))
    plt.barh(range(len(feature_importance_df)), feature_importance_df['importance'], color='teal')
    plt.yticks(range(len(feature_importance_df)), feature_importance_df['feature'])
    plt.xlabel('Importancia', fontsize=12)
    plt.title(f'Top {top_n} Features Más Importantes - {best_model_name}', 
             fontsize=14, fontweight='bold')
    plt.gca().invert_yaxis()
    plt.grid(axis='x', alpha=0.3)
    plt.tight_layout()
    plt.savefig(f'../../feature_importance_{best_model_name.replace(" ", "_")}.png', 
               dpi=300, bbox_inches='tight')
    print(f'✅ Gráfico guardado: feature_importance_{best_model_name.replace(" ", "_")}.png')
    plt.show()
    
    print(f'\nTop {top_n} Features:')
    print(feature_importance_df.to_string(index=False))
else:
    print('⚠️ El modelo seleccionado no soporta feature importance')

---
## 8. GUARDAR MEJOR MODELO

In [ ]:
print('\n' + '='*80)
print('GUARDAR MEJOR MODELO')
print('='*80)

In [ ]:
# Guardar mejor modelo con metadata
best_model = best_result['model']
best_metrics = best_result['test_metrics']

# Cargar preprocessor si está disponible
try:
    preprocessor = joblib.load('../../preprocessor.pkl')
except:
    preprocessor = None

# Obtener nombres de features
try:
    feature_names = list(pd.read_csv('../../X_train_transformed.csv', nrows=0).columns)
except:
    feature_names = None

model_package = {
    'model': best_model,
    'model_name': best_model_name,
    'metrics': best_metrics,
    'preprocessor': preprocessor,
    'feature_names': feature_names,
    'timestamp': datetime.now().isoformat(),
    'version': '1.0'
}

filename = '../../best_model.pkl'
joblib.dump(model_package, filename)

print(f'\n✅ Modelo guardado: {filename}')
print(f'   Modelo: {best_model_name}')
print(f'   F1-Score: {best_metrics["f1"]:.4f}')
print(f'   ROC-AUC: {best_metrics.get("roc_auc", "N/A")}')

---
## 9. RESUMEN FINAL

In [ ]:
print('\n' + '='*80)
print('RESUMEN FINAL - FASE 3')
print('='*80)

print('\n✅ FASE 3 COMPLETADA EXITOSAMENTE')
print(f'\n📊 Resumen:')
print(f'  1. Modelos entrenados: {len(results)}')
print(f'  2. Mejor modelo: {best_model_name}')
print(f'  3. F1-Score: {best_result["test_metrics"]["f1"]:.4f}')
print(f'  4. ROC-AUC: {best_result["test_metrics"].get("roc_auc", "N/A")}')
print(f'  5. Accuracy: {best_result["test_metrics"]["accuracy"]:.4f}')

print('\n📁 Archivos generados:')
print('  - best_model.pkl')
print('  - model_comparison_metrics.png')
print('  - model_comparison_roc_auc.png')
print('  - roc_curves_comparison.png')
print('  - confusion_matrices.png')
if hasattr(best_result['model'], 'feature_importances_'):
    print(f'  - feature_importance_{best_model_name.replace(" ", "_")}.png')

print('\n' + '='*80)
print('🎉 FASE 3 COMPLETADA - LISTO PARA FASE 4')
print('='*80)

---
## 10. VERIFICACIÓN

In [ ]:
# Verificar que el modelo se puede cargar
try:
    loaded_model = joblib.load('../../best_model.pkl')
    print('✅ Modelo cargado correctamente')
    print(f'   Modelo: {loaded_model["model_name"]}')
    print(f'   F1-Score: {loaded_model["metrics"]["f1"]:.4f}')
    print(f'   Timestamp: {loaded_model["timestamp"]}')
except Exception as e:
    print(f'❌ Error al cargar modelo: {str(e)}')